In [1]:
from __future__ import print_function
import numpy as np
import os
import sys
from six.moves import cPickle as pickle

In [2]:
import tensorflow as tf
from decimal import Decimal

In [3]:
HUMAN_MOTIF_VARIANTS = [
    'AATAAA',
    'ATTAAA',
    'AAAAAG',
    'AAGAAA',
    'TATAAA',
    'AATACA',
    'AGTAAA',
    'ACTAAA',
    'GATAAA',
    'CATAAA',
    'AATATA',
    'AATAGA'
]

In [4]:
POS_PATH = 'data/human/dragon_polyA_data/positive5fold/'
NEG_PATH = 'data/human/dragon_polyA_data/negatives5fold/'
# POS_PATH = 'human_data/omni_polyA_data/positive/'
# NEG_PATH = 'human_data/omni_polyA_data/negative/'
BATCH_SIZE = 64
PATCH_SIZE = 10
DEPTH = 16
NUM_HIDDEN = 64
SEQ_LEN = 206 + 2*PATCH_SIZE-2
NUM_CHANNELS = 4
NUM_LABELS = 2
NUM_EPOCHS = 200
NUM_FOLDS = 5
HYPER_DICT = None

In [5]:
#用户可以选择输入的目录等参数
tf.app.flags.DEFINE_string('train_dir', None, 'Directory where checkpoints are written to.') 
tf.app.flags.DEFINE_integer('training_job_index', 0, 'index of training result for logging')
tf.app.flags.DEFINE_string('training_result_dir', None, 'The file which the training result is written to')

FLAGS = tf.app.flags.FLAGS

In [6]:
#这一块其实用不到，不用管
# Disable print
def block_print():
    sys.stdout = open(os.devnull, 'w')


# Restore print
def enable_print():
    sys.stdout = sys.__stdout__

In [7]:
#从文件路径取到文件
def get_motif_data(data_root, label):
    data = {}
    labels = {}
    for motif in HUMAN_MOTIF_VARIANTS:   #把数据按照motif分类放好
        data[motif] = []
        for data_file in os.listdir(data_root):
            if motif in data_file:
                data_path = os.path.join(data_root, data_file)
                with open(data_path, 'r') as f:
                    alphabet = np.array(['A','G','T','C'])
                    for line in f:
                        line = list(line.strip('\n'))
                        seq = np.array(line, dtype = '|U1').reshape(-1,1)
                        seq_data = (seq == alphabet).astype(np.float32) 
                        data[motif].append(seq_data)                       #转化成one-hot表示法 如1 1 0 1
        data[motif] = np.stack(data[motif]).reshape([-1,206,1,4])
        if label:
            labels[motif] = np.zeros(data[motif].shape[0])      
        else:
            labels[motif] = np.ones(data[motif].shape[0])
    return data, labels

In [8]:
def shuffle(dataset, labels, randomState = None):     #把数据随机打乱
    if randomState is None:
        permutation = np.random.permutation(labels.shape[0])
    else:
        permutation = randomState.permutation(labels.shape[0])
    shuffled_data = dataset[permutation,:,:]
    shuffled_labels = labels[permutation]
    return shuffled_data, shuffled_labels

In [9]:
def produce_motif_dataset(num_folds, pos_path, neg_path, seed=0):     #调用上面两个函数，放好数据
    pos_data, pos_labels = get_motif_data(pos_path, True)
    neg_data, neg_labels = get_motif_data(neg_path, False)
    randomState = np.random.RandomState(seed)
    for motif in HUMAN_MOTIF_VARIANTS:
        pos_data[motif],pos_labels[motif] = shuffle(pos_data[motif], pos_labels[motif], randomState)
        neg_data[motif],neg_labels[motif] = shuffle(neg_data[motif], neg_labels[motif], randomState)
        print('Positive %s:'%motif, pos_data[motif].shape, pos_labels[motif].shape)
        print('Negative %s:'%motif, neg_data[motif].shape, neg_labels[motif].shape)
    return pos_data, pos_labels, neg_data, neg_labels

In [10]:
def gen_hyper_dict(hyper_dict=None):   #随机设置超参数
    def rand_log(a,b):
        x = np.random.sample()
        return 10.0 ** ((np.log10(b) - np.log10(a)) * x + np.log10(a))
    
    def rand_sqrt(a,b):
        x = np.random.sample()
        return (b - a) * np.sqrt(x) + a
    
    if hyper_dict is None:
        hyper_dict = {
            'tf_learning_rate': rand_log(.0005, .05),
            'tf_momentum': rand_sqrt(.95,.99),
            'tf_motif_init_weight': rand_log(1e-2,10),
            'tf_fc_init_weight': rand_log(1e-2,10),
            'tf_motif_weight_decay': rand_log(1e-5, 1e-3),
            'tf_fc_weight_decay': rand_log(1e-5, 1e-3),
            'tf_keep_prob': np.random.choice([.5, .75, 1.0]),
            'tf_ngroups': np.random.choice([2,4,8])
        }
    #for k, v in hyper_dict.items():
    #    print("%s: %.2e"%(k, Decimal(v)))
    #    print()
    return hyper_dict

In [11]:
#将数据分成五份，用于交叉验证，并设置train，valid，test数据集
def data_split(pos_data, pos_labels, neg_data, neg_labels, num_folds, split):
    pos_data_folds = np.array_split(pos_data, num_folds)
    neg_data_folds = np.array_split(neg_data, num_folds)
    pos_label_folds = np.array_split(pos_labels, num_folds)
    neg_label_folds = np.array_split(neg_labels, num_folds)
    
    train_pos_data = np.concatenate([pos_data_folds[i] for i in split['train']], axis = 0)
    train_pos_labels = np.concatenate([pos_label_folds[i] for i in split['train']], axis=0)
    valid_pos_data = np.concatenate([pos_data_folds[i] for i in split['valid']], axis=0)
    valid_pos_labels = np.concatenate([pos_label_folds[i] for i in split['valid']], axis=0)
    
    train_neg_data = np.concatenate([neg_data_folds[i] for i in split['train']], axis=0)
    train_neg_labels = np.concatenate([neg_label_folds[i] for i in split['train']], axis=0)
    valid_neg_data = np.concatenate([neg_data_folds[i] for i in split['valid']], axis=0)
    valid_neg_labels = np.concatenate([neg_label_folds[i] for i in split['valid']], axis=0)
    
    train_data = np.concatenate((train_pos_data, train_neg_data), axis=0)
    valid_data = np.concatenate((valid_pos_data, valid_neg_data), axis=0)
    train_labels = np.concatenate((train_pos_labels, train_neg_labels), axis=0)
    valid_labels = np.concatenate((valid_pos_labels, valid_neg_labels), axis=0)
    
    data = {}
    data['train_dataset'], data['train_labels'] = shuffle(train_data, train_labels)
    data['valid_dataset'], data['valid_labels'] = shuffle(valid_data, valid_labels)
    
    if 'test' in split:
        test_pos_data = np.concatenate([pos_data_folds[i] for i in split['test']], axis=0)
        test_pos_labels = np.concatenate([pos_label_folds[i] for i in split['test']], axis=0)
        test_neg_data = np.concatenate([neg_data_folds[i] for i in split['test']], axis=0)
        test_neg_labels = np.concatenate([neg_label_folds[i] for i in split['test']], axis=0)
        test_data = np.concatenate((test_pos_data, test_neg_data), axis=0)
        test_labels = np.concatenate((test_pos_labels, test_neg_labels), axis=0)
        data['test_dataset'], data['test_labels'] = shuffle(test_data, test_labels)
    
    return data

In [12]:
#将每一个motif的数据分开，设置train，valid，test数据集
def motif_data_split(pos_data, pos_labels, neg_data, neg_labels, num_folds, split):
    motif_data = {}
    for motif in HUMAN_MOTIF_VARIANTS:
        motif_data[motif] = data_split(pos_data[motif], pos_labels[motif], neg_data[motif], neg_labels[motif], num_folds, split)
    
    train_data = np.concatenate([motif_data[motif]['train_dataset'] for motif in HUMAN_MOTIF_VARIANTS], axis=0)
    valid_data = np.concatenate([motif_data[motif]['valid_dataset'] for motif in HUMAN_MOTIF_VARIANTS], axis=0)
    test_data = np.concatenate([motif_data[motif]['test_dataset'] for motif in HUMAN_MOTIF_VARIANTS], axis=0)
    train_labels = np.concatenate([motif_data[motif]['train_labels'] for motif in HUMAN_MOTIF_VARIANTS], axis=0)
    valid_labels = np.concatenate([motif_data[motif]['valid_labels'] for motif in HUMAN_MOTIF_VARIANTS], axis=0)
    test_labels = np.concatenate([motif_data[motif]['test_labels'] for motif in HUMAN_MOTIF_VARIANTS], axis=0)
    
    data = {}
    data['train_dataset'], data['train_labels'] = shuffle(train_data, train_labels)
    data['valid_dataset'], data['valid_labels'] = shuffle(valid_data, valid_labels)
    data['test_dataset'], data['test_labels'] = shuffle(test_data, test_labels)
    
    data['motif_dataset'] = {motif: {} for motif in HUMAN_MOTIF_VARIANTS}
    for motif in HUMAN_MOTIF_VARIANTS:
        data['motif_dataset'][motif]['test_dataset'] = motif_data[motif]['test_dataset']
        data['motif_dataset'][motif]['test_labels'] = motif_data[motif]['test_labels']
        
    return data

In [13]:
#填充数据用于卷积
def pad_dataset(dataset, labels):
    new_dataset = np.ones([dataset.shape[0], dataset.shape[1] + 2*PATCH_SIZE-2, dataset.shape[2], dataset.shape[3]], dtype = np.float32) * 0.25
    new_dataset[:,PATCH_SIZE-1:-(PATCH_SIZE-1),:,:] = dataset
    labels = (np.arange(NUM_LABELS) == labels[:,None]).astype(np.float32)
    return new_dataset, labels

In [14]:
#计算预测精确度
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels,1)) / predictions.shape[0])

In [15]:
def train(dataset, hyper_dict):
    graph = tf.Graph()
    
    with graph.as_default():
        
        #取到参数
        tf_learning_rate = hyper_dict['tf_learning_rate']
        tf_momentum = hyper_dict['tf_momentum']
        tf_motif_init_weight = hyper_dict['tf_motif_init_weight']
        tf_fc_init_weight = hyper_dict['tf_fc_init_weight']
        tf_motif_weight_decay = hyper_dict['tf_motif_weight_decay']
        tf_fc_weight_decay = hyper_dict['tf_fc_weight_decay']
        tf_keep_prob = hyper_dict['tf_keep_prob']
        tf_ngroups = hyper_dict['tf_ngroups']
        
        #原始数据
        tf_train_dataset = tf.placeholder(tf.float32, shape = (BATCH_SIZE, SEQ_LEN, 1, NUM_CHANNELS))
        tf_train_labels = tf.placeholder(tf.float32, shape = (BATCH_SIZE, NUM_LABELS))
        tf_train_valid_dataset = tf.constant(dataset['train_dataset'])
        tf_valid_dataset = tf.constant(dataset['valid_dataset'])
        tf_test_dataset = tf.constant(dataset['test_dataset'])
        tf_motif_test_dataset = {}
        for motif in HUMAN_MOTIF_VARIANTS:
            tf_motif_test_dataset[motif] = tf.constant(dataset['motif_dataset'][motif]['test_dataset'])
        
        #各层参数
        conv_weights = tf.Variable(tf.truncated_normal([PATCH_SIZE, 1, NUM_CHANNELS, DEPTH], stddev=tf_motif_init_weight))
        conv_biases = tf.Variable(tf.zeros([DEPTH]))
        layer1_weights = tf.Variable(tf.truncated_normal([21*DEPTH, NUM_HIDDEN], stddev = tf_fc_init_weight))
        layer1_biases = tf.Variable(tf.constant(1.0, shape=[NUM_HIDDEN]))
        layer2_weights = tf.Variable(tf.truncated_normal([NUM_HIDDEN,NUM_LABELS], stddev=tf_fc_init_weight))
        layer2_biases = tf.Variable(tf.constant(1.0, shape=[NUM_LABELS]))
        
        weights = {}
        weights['conv_weights'] = conv_weights
        weights['conv_biases'] = conv_biases
        weights['layer1_weights'] = layer1_weights
        weights['layer1_biases'] = layer1_biases
        weights['layer2_weights'] = layer2_weights
        weights['layer2_biases'] = layer2_biases
        
        #主要的模型
        def model(data, drop=True):
            conv = tf.nn.conv2d(data, conv_weights, [1,1,1,1], padding='VALID')
            conv = tf.reshape(conv, [-1, 215, 1, DEPTH//tf_ngroups, tf_ngroups])
            mu, var = tf.nn.moments(conv, [1,2,3], keep_dims=True)
            conv = (conv - mu) / tf.sqrt(var + 1e-12)
            conv = tf.reshape(conv, [-1, 215, 1, DEPTH])
            hidden = tf.nn.relu(conv + conv_biases)
            hidden = tf.nn.max_pool(hidden, [1,10,1,1], [1,10,1,1], padding = "VALID")
            shape = hidden.get_shape().as_list()
            motif_score = tf.reshape(hidden, [shape[0], shape[1]*DEPTH])
            if drop:
                hidden_nodes = tf.nn.dropout(tf.nn.relu(tf.matmul(motif_score, layer1_weights) + layer1_biases), tf_keep_prob)
            else:
                hidden_nodes = tf.nn.relu(tf.matmul(motif_score, layer1_weights) + layer1_biases)
            return tf.matmul(hidden_nodes, layer2_weights) + layer2_biases
        logits = model(tf_train_dataset)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + tf_fc_weight_decay * (tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer2_weights)) + tf_motif_weight_decay*tf.nn.l2_loss(conv_weights)
        
        #optimizer， 实现梯度下降
        global_step = tf.Variable(0, trainable=False)
        stepOp = tf.assign_add(global_step, 1).op
        learning_rate = tf.train.exponential_decay(tf_learning_rate, global_step, 3000, 0.96)
        optimizer = tf.train.MomentumOptimizer(learning_rate, tf_momentum).minimize(loss)
        
        #predictions 预测结果
        train_prediction = tf.nn.softmax(model(tf_train_valid_dataset, drop = False))
        valid_prediction = tf.nn.softmax(model(tf_valid_dataset, drop = False))
        test_prediction = tf.nn.softmax(model(tf_test_dataset, drop = False))
        motif_test_prediction = {}
        for motif in HUMAN_MOTIF_VARIANTS:
            motif_test_prediction[motif] = tf.nn.softmax(model(tf_motif_test_dataset[motif], drop = False))
            
    train_resuts = []
    valid_results = []
    test_results = []
    motif_test_results = {motif: [] for motif in HUMAN_MOTIF_VARIANTS}
    save_weights = []
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        train_dataset = dataset['train_dataset']
        train_labels = dataset['train_labels']
        np.random.seed()
        print("Initialized")
        print('Training accuracy at begining: %.1f%%' % accuracy(train_prediction.eval(), train_labels))
        print('Validation accuracy at the beginning: %.1f%%' % accuracy(valid_prediction.eval(), dataset['valid_labels']))
        for epoch in range(NUM_EPOCHS):                         #对于每一个epoch
            permutation = np.random.permutation(train_labels.shape[0])
            shuffled_dataset = train_dataset[permutation,:,:]
            shuffled_labels = train_labels[permutation,:]
            for step in range(shuffled_labels.shape[0]//BATCH_SIZE):     #对于一个epoch里每一个batch的数据
                offset = step * BATCH_SIZE
                batch_data = train_dataset[offset:(offset + BATCH_SIZE),:,:,:]
                batch_labels = train_labels[offset:(offset + BATCH_SIZE), :]
                feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
                _, l = session.run([optimizer,loss], feed_dict = feed_dict)      #调用optimizer，实现梯度下降
                session.run(stepOp)
            
            train_resuts.append(accuracy(train_prediction.eval(), train_labels))    #计算结果并储存
            valid_pred = valid_prediction.eval()
            valid_results.append(accuracy(valid_pred, dataset['valid_labels']))
            test_pred = test_prediction.eval()
            test_results.append(accuracy(test_pred, dataset['test_labels']))
            for motif in HUMAN_MOTIF_VARIANTS:
                motif_test_pred = motif_test_prediction[motif].eval()
                motif_test_results[motif].append(accuracy(motif_test_pred, dataset['motif_dataset'][motif]['test_labels']))
            print('Training accuracy at epoch %d: %.1f%%' % (epoch, train_resuts[-1]))
            print('Validation accuracy: %.1f%%' % valid_results[-1])
            
            #early stopping  #不能取到更好的结果时及时停止
            if epoch > 10 and valid_results[-11] > max(valid_results[-10:]):
                train_resuts = train_resuts[:-10]
                valid_results = valid_results[:-10]
                test_results = test_results[:-10]
                motif_test_results = {motif: motif_test_results[motif][:-10] for motif in HUMAN_MOTIF_VARIANTS}
                return train_resuts, valid_results, test_results, motif_test_results, save_weights[0]
            
            #model saving  #保存最新的参数
            sw = {}
            for k in weights:
                sw[k] = weights[k].eval()
            if epoch < 10:
                save_weights.append(sw)
            else:
                save_weights.append(sw)
                save_weights.pop(0)
            
    return train_resuts, valid_results, test_results, motif_test_results, save_weights[-1]
        

In [16]:
def main(_):
    # block_print()

    hyper_dict = gen_hyper_dict(HYPER_DICT)
    pos_data, pos_labels, neg_data, neg_labels = produce_motif_dataset(NUM_FOLDS, POS_PATH, NEG_PATH)

    # Cross validate
    train_accuracy_split = []
    valid_accuracy_split = []
    test_accuracy_split = []
    motif_test_accuracy_split = {motif: [] for motif in HUMAN_MOTIF_VARIANTS}
    
    for i in range(NUM_FOLDS):
        split = {                                                            #交叉验证时split数据的方法
            'train':[(i + j) % NUM_FOLDS for j in range(NUM_FOLDS-2)],
            'valid':[(i + NUM_FOLDS - 2) % NUM_FOLDS],
            'test': [(i + NUM_FOLDS - 1) % NUM_FOLDS]
        }
        save = motif_data_split(pos_data, pos_labels, neg_data, neg_labels, NUM_FOLDS, split)
        dataset = {}
        dataset['train_dataset'], dataset['train_labels'] = pad_dataset(save['train_dataset'], save['train_labels'])   #pad数据
        dataset['valid_dataset'], dataset['valid_labels'] = pad_dataset(save['valid_dataset'], save['valid_labels'])
        dataset['test_dataset'], dataset['test_labels'] = pad_dataset(save['test_dataset'], save['test_labels'])
        dataset['motif_dataset'] = {}
        for motif in HUMAN_MOTIF_VARIANTS:
            dataset['motif_dataset'][motif] = {}
            dataset['motif_dataset'][motif]['test_dataset'], dataset['motif_dataset'][motif]['test_labels'] = pad_dataset(save['motif_dataset'][motif]['test_dataset'], save['motif_dataset'][motif]['test_labels'])
        train_resuts, valid_results, test_results, motif_test_results, save_weights = train(dataset, hyper_dict)
        print("\nbest valid epoch: %d"%(len(train_resuts)-1))
        print("Training accuracy: %.2f%%"%train_resuts[-1])
        print("Test accuracy: %.2f%%"%test_results[-1])
        print("Validation accuracy: %.2f%%"%valid_results[-1])
        for motif in HUMAN_MOTIF_VARIANTS:
            print('*%s* accuracy: %.1f%%' % (motif, motif_test_results[motif][-1]))

        #Dump model
        if FLAGS.train_dir is not None:                                                   #保存weights到本地文件
            with open(os.path.join(FLAGS.train_dir, 'cv%d_model.pkl'%i), "wb") as f:
                pickle.dump(save_weights, f, 2)
                
        train_accuracy_split.append(train_resuts[-1])                               #保存预测的accuracy
        valid_accuracy_split.append(valid_results[-1])
        test_accuracy_split.append(test_results[-1])
        for motif in HUMAN_MOTIF_VARIANTS:
            motif_test_accuracy_split[motif].append(motif_test_results[motif][-1])
        
    train_accuracy = np.mean(train_accuracy_split)                         #取五次的均值作为最后的准确度
    valid_accuracy = np.mean(valid_accuracy_split)
    test_accuracy = np.mean(test_accuracy_split)
    motif_test_accuracy = {}
    for motif in HUMAN_MOTIF_VARIANTS:
        motif_test_accuracy[motif] = np.mean(motif_test_accuracy_split[motif])
    print('\n\n########################\nFinal result:')
    print('Training accuracy: %.1f%%' % (train_accuracy))
    print('Validation accuracy: %.1f%%' % (valid_accuracy))
    print('Test accuracy: %.1f%%' % (test_accuracy ))
    for motif in HUMAN_MOTIF_VARIANTS:
        print('*%s* accuracy: %.1f%%' % (motif, motif_test_accuracy[motif]))

    if FLAGS.training_result_dir is not None:
        with open(os.path.join(FLAGS.training_result_dir, 'result.pkl'), 'wb') as f:         #保存训练的参数到本地
            hyper_dict['train_accuracy'] = train_accuracy
            hyper_dict['valid_accuracy'] = valid_accuracy
            hyper_dict['test_accuracy'] = test_accuracy
            hyper_dict['motif_test_accuracy'] = motif_test_accuracy
            pickle.dump(hyper_dict, f, 2)
                   
    

In [17]:
if __name__ == '__main__':
    tf.app.run()

Positive AATAAA: (2595, 206, 1, 4) (2595,)
Negative AATAAA: (2595, 206, 1, 4) (2595,)
Positive ATTAAA: (1200, 206, 1, 4) (1200,)
Negative ATTAAA: (1200, 206, 1, 4) (1200,)
Positive AAAAAG: (615, 206, 1, 4) (615,)
Negative AAAAAG: (615, 206, 1, 4) (615,)
Positive AAGAAA: (625, 206, 1, 4) (625,)
Negative AAGAAA: (625, 206, 1, 4) (625,)
Positive TATAAA: (390, 206, 1, 4) (390,)
Negative TATAAA: (390, 206, 1, 4) (390,)
Positive AATACA: (440, 206, 1, 4) (440,)
Negative AATACA: (440, 206, 1, 4) (440,)
Positive AGTAAA: (335, 206, 1, 4) (335,)
Negative AGTAAA: (335, 206, 1, 4) (335,)
Positive ACTAAA: (345, 206, 1, 4) (345,)
Negative ACTAAA: (345, 206, 1, 4) (345,)
Positive GATAAA: (230, 206, 1, 4) (230,)
Negative GATAAA: (230, 206, 1, 4) (230,)
Positive CATAAA: (205, 206, 1, 4) (205,)
Negative CATAAA: (205, 206, 1, 4) (205,)
Positive AATATA: (205, 206, 1, 4) (205,)
Negative AATATA: (205, 206, 1, 4) (205,)
Positive AATAGA: (185, 206, 1, 4) (185,)
Negative AATAGA: (185, 206, 1, 4) (185,)
Instruct

Training accuracy at epoch 1: 82.1%
Validation accuracy: 81.7%
Training accuracy at epoch 2: 85.7%
Validation accuracy: 84.1%
Training accuracy at epoch 3: 88.8%
Validation accuracy: 87.3%
Training accuracy at epoch 4: 90.7%
Validation accuracy: 88.3%
Training accuracy at epoch 5: 90.6%
Validation accuracy: 88.7%
Training accuracy at epoch 6: 90.2%
Validation accuracy: 87.8%
Training accuracy at epoch 7: 91.0%
Validation accuracy: 88.1%
Training accuracy at epoch 8: 93.0%
Validation accuracy: 89.5%
Training accuracy at epoch 9: 93.4%
Validation accuracy: 89.5%
Training accuracy at epoch 10: 90.9%
Validation accuracy: 87.4%
Training accuracy at epoch 11: 92.6%
Validation accuracy: 89.0%
Training accuracy at epoch 12: 93.7%
Validation accuracy: 89.1%
Training accuracy at epoch 13: 93.2%
Validation accuracy: 88.3%
Training accuracy at epoch 14: 93.0%
Validation accuracy: 88.4%
Training accuracy at epoch 15: 92.9%
Validation accuracy: 88.2%
Training accuracy at epoch 16: 93.2%
Validation a

SystemExit: 

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
